In [20]:
import dotenv
from pathlib import Path
import pandas as pd
from datetime import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [21]:
envPath = ".env"

Path(envPath).exists()
if not dotenv.load_dotenv(envPath):
    raise RuntimeError(f"Environment Variables not found in {envPath}")

In [22]:
# Conectarse con permisos de lectura
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager = auth_manager)

In [23]:
# playlists = sp.user_playlists(user='spotify')
playlists = sp.user_playlists(user='domnes42')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print(f"{i + 1 + playlists['offset']:4d} {playlist['uri']} {playlist['name']}")
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

   1 spotify:playlist:0m5m7XBftn0kbR8nJTy81z Bandas Sonoras
   2 spotify:playlist:1tkHhAAq6yvcwTtxEJlw8E Gorillaz
   3 spotify:playlist:7BSqrCtujKvQh9KPKk0Gqc Canciones chulindangas que escuchaba hace unos años
   4 spotify:playlist:03fpAdWRT8FuAXpJ4Zgmqr Pa la dusha (cantables)
   5 spotify:playlist:7tWw42qiQ5HONQFGXxx5o5 Chilling this much feels illegal
   6 spotify:playlist:3sFQurSTJr6Ow1zSuIsEOP Watch later
   7 spotify:playlist:3MTrGsLcMu55IroZRVrBwn Pa la dusha
   8 spotify:playlist:6JZjlU7AQUOfRx4flxhK71 Coche cochecito


In [24]:
def get_playlist_tracks(sp: spotipy.Spotify, playlist_url: str) -> pd.DataFrame:
    """Extract tracks from a url (only read permissions needed)"""
    playlist_name = sp.playlist(playlist_id = playlist_url, fields = "name")["name"]

    # Create a list accounting for the offset
    rows, offset = [], 0
    while True:
            
        raw_tracks  =   sp.playlist_tracks(
            playlist_id =   playlist_url,
            limit       =   100,
            offset      =   offset,
            fields      =   "items(added_at,track(id,name,artists(name),album(name),duration_ms,external_ids(isrc))),next"
        )

        for row in raw_tracks["items"]:
            date = str(row["added_at"])
            info = row["track"]
            track_info = {
                "origin":           "spotify",
                "playlist_name":    playlist_name,
                "playlist_url":     playlist_url,
                "track_name":       info["name"],
                "track_id":         info["id"],
                "artist":           ", ".join([artist["name"] for artist in info["artists"]]),
                "album":            info["album"]["name"],
                "duration_ms":      info["duration_ms"],
                "added_at":         date
            }

            rows.append(track_info)
        
        if not raw_tracks["next"]: break # exit if there is no more tracks to add
        offset += 100
        
    return pd.DataFrame(rows)

In [25]:
tracks = get_playlist_tracks(sp, r"https://open.spotify.com/playlist/0mSjZmnhxJLUp7wzHTtibc")
today = datetime.now().strftime("%Y-%m-%d_%H%M%S")
user = sp.current_user()["id"]

output_dir = Path("exports")
output_dir.mkdir(parents = True, exist_ok = True)

tracks.to_csv(output_dir / f"spotify_{today}_{user}.csv", index = False)
tracks.head()

HTTP Error for GET to https://api.spotify.com/v1/me/ with Params: {} returned 401 due to Valid user authentication required


SpotifyException: http status: 401, code: -1 - https://api.spotify.com/v1/me/:
 Valid user authentication required, reason: None

In [28]:
# Conectarse con permiso de escritura de playlists
sp = spotipy.Spotify(
    auth_manager = spotipy.SpotifyOAuth(
        scope = "playlist-modify-private playlist-modify-public playlist-read-private playlist-read-collaborative user-library-read",
        cache_path=".cache-playlists",
        show_dialog=True
    )
)

In [ ]:
# playlist_name,playlist_id,track_title,artist,album,isrc,duration_ms,added_at

newPlaylist = sp.user_playlist_create(
    user = sp.current_user()["id"],
    name = "prueba_spotipy",
    public = False,
    description = "Descripción de la prueba de Spotipy"
)

newPlaylist

{'collaborative': False,
 'description': 'Descripción de la prueba de Spotipy',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0mSjZmnhxJLUp7wzHTtibc'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0mSjZmnhxJLUp7wzHTtibc',
 'id': '0mSjZmnhxJLUp7wzHTtibc',
 'images': [],
 'primary_color': None,
 'name': 'prueba_spotipy',
 'type': 'playlist',
 'uri': 'spotify:playlist:0mSjZmnhxJLUp7wzHTtibc',
 'owner': {'href': 'https://api.spotify.com/v1/users/domnes42',
  'id': 'domnes42',
  'type': 'user',
  'uri': 'spotify:user:domnes42',
  'display_name': None,
  'external_urls': {'spotify': 'https://open.spotify.com/user/domnes42'}},
 'public': False,
 'snapshot_id': 'AAAAcXA9lmkO+snwFM6xumO3O+3w8fNC',
 'tracks': {'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'href': 'https://api.spotify.com/v1/playlists/0mSjZmnhxJLUp7wzHTtibc/tracks',
  'total': 0,
  'items': []}}

In [ ]:
tracks_id = [track["track"]["id"] for track in tracks["items"]]

sp.user_playlist_add_tracks(
    user = sp.current_user()["id"],
    playlist_id = newPlaylist["id"],
    tracks = tracks_id
)

{'snapshot_id': 'AAAAAjnXSgqfJkNiL6XVWnUEH/tp9R8R'}

In [33]:
sp.current_user_saved_tracks()["items"][0]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [34]:
def get_credentials(scope: str) -> spotipy.Spotify:
    # Get credentials
    # TODO: from .env to actual writing your info. Maybe looking for a .env, if not found, ask for the data.
    envPath = ".env"

    Path(envPath).exists()
    if not dotenv.load_dotenv(envPath):
        raise RuntimeError(f"Environment Variables not found in {envPath}")

    # Connect with read permissions
    sp = spotipy.Spotify(
        auth_manager = spotipy.SpotifyOAuth(
            scope = scope,
            cache_path=".cache-playlists",
            show_dialog=True
        )
    )
    return sp

In [36]:
sp = spotipy.Spotify(
        auth_manager = spotipy.SpotifyOAuth(
            cache_path=".cache-playlists",
            show_dialog=True
        )
    )

In [37]:
print(sp.auth_manager.get_cached_token()["scope"])  # must include playlist-modify-private

playlist-modify-private playlist-modify-public playlist-read-private playlist-read-collaborative user-library-read


In [38]:
from datetime import datetime

me = sp.current_user()
pl = sp.user_playlist_create(
    user=me["id"],
    name="aa",
    public=False,
    description=f"Generated from CSV Backup. Upload: {datetime.now().strftime('%d-%m-%Y, %H:%M:%S')}"
)
pl_full = sp.playlist(pl["id"], fields="id,name,public,owner(id)")
print("public?", pl_full["public"])  # expect False


public? True


In [40]:
sp.user_playlist_change_details(me["id"], pl["id"], public=False)

In [41]:
me = sp.current_user()
pl = sp.user_playlist_create(
    me["id"],
    "playlist_name",
    public=False,
    description="…"
)
sp.user_playlist_change_details(me["id"], pl["id"], public=False)

pl_full = sp.playlist(pl["id"], fields="id,name,public,owner(id)")
print("owner:", pl_full["owner"]["id"], "public:", pl_full["public"])


owner: domnes42 public: True
